In [1]:
# import relevant libraries

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# load all csv files with headers, except ftp-write, which does not have column names
df_back = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back.csv")
df_buffer_overflow = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_BufferOverflow.csv")
df_guess_password = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_GuessPassword.csv")
df_neptune = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Neptune.csv")
df_nmap = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_NMap.csv")
df_normal = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Normal.csv")
df_port_sweep = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_PortSweep.csv")
df_rootkit = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_RootKit.csv")
df_satan = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Satan.csv")
df_smurf = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_Smurf.csv")

In [3]:
# load ftp-write without headers
df_ftp_write = pd.read_csv(r"/Users/adityamxr/Desktop/Cybersecurity Project/data/Data_of_Attack_Back_FTPWrite.csv", header = None)

In [4]:
# verify import

df_back.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.001,0.001,0.1,0,0.100,0,0.0,0.0,0.0,0.0
1,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.002,0.002,0.1,0,0.050,0,0.0,0.0,0.0,0.0
2,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.003,0.003,0.1,0,0.033,0,0.0,0.0,0.0,0.0
3,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.004,0.004,0.1,0,0.025,0,0.0,0.0,0.0,0.0
4,0.0,0,0,0.0,0.5454,0.08314,0,0,0,0.2,...,0.005,0.005,0.1,0,0.020,0,0.0,0.0,0.0,0.0


In [5]:
# assign correct column names to df_ftp_write
df_ftp_write.columns = df_back.columns

In [6]:
# verify 
df_ftp_write.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0026,0,0.07,0,0.00116,0.00451,0,0,0.0,0.2,...,0.001,0.001,0.10,0.0,0.10,0.000,0,0,0,0
1,0.0134,0,0.34,0,0.00100,0.39445,0,0,0.2,0.0,...,0.002,0.001,0.05,0.1,0.05,0.000,0,0,0,0
2,0.0000,0,0.14,0,0.00613,0.00000,0,0,0.0,0.0,...,0.001,0.084,0.10,0.0,0.10,0.002,0,0,0,0
3,0.0000,0,0.14,0,0.00000,0.00005,0,0,0.0,0.0,...,0.002,0.085,0.10,0.0,0.10,0.002,0,0,0,0
4,0.0032,0,0.07,0,0.00104,0.00449,0,0,0.0,0.2,...,0.001,0.001,0.10,0.0,0.10,0.000,0,0,0,0


In [7]:
# check dataframe info for basic EDA
dfs = [df_back, df_buffer_overflow, df_ftp_write, df_guess_password, 
                      df_neptune, df_nmap, df_normal, df_port_sweep, df_rootkit, 
                      df_satan, df_smurf]

for df in dfs:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   duration                      968 non-null    float64
 1    protocol_type                968 non-null    int64  
 2    service                      968 non-null    int64  
 3    flag                         968 non-null    float64
 4    src_bytes                    968 non-null    float64
 5    dst_bytes                    968 non-null    float64
 6    land                         968 non-null    int64  
 7    wrong_fragment               968 non-null    int64  
 8    urgent                       968 non-null    int64  
 9    hot                          968 non-null    float64
 10   num_failed_logins            968 non-null    int64  
 11   logged_in                    968 non-null    float64
 12   num_compromised              968 non-null    float64
 13   root

**No nulls are present in any dataframe. Proceed with appending the dataframes.**

In [8]:
# add an attack column in each df, and label the rows with the respective attack type

df_back['attack'] = 'back'
df_buffer_overflow['attack'] = 'buffer overflow'
df_ftp_write['attack'] = 'ftpwrite'
df_guess_password['attack'] = 'guess password'
df_neptune['attack'] = 'neptune'
df_nmap['attack'] = 'nmap'
df_normal['attack'] = 'normal'
df_port_sweep['attack'] = 'port sweep'
df_rootkit['attack'] = 'rootkit'
df_satan['attack'] = 'satan'
df_smurf['attack'] = 'smurf'

In [9]:
# verify

df_smurf.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.001,0.001,0.002,0.001,0,0.0,0,0.0,0,smurf
1,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.002,0.001,0.002,0.001,0,0.0,0,0.0,0,smurf
2,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.003,0.002,0.002,0.002,0,0.0,0,0.0,0,smurf
3,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.004,0.003,0.002,0.003,0,0.0,0,0.0,0,smurf
4,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.005,0.003,0.002,0.003,0,0.0,0,0.0,0,smurf


In [10]:
# append the dataframes into one final_df

final_df = pd.concat([df_back, df_buffer_overflow, df_ftp_write, df_guess_password, 
                      df_neptune, df_nmap, df_normal, df_port_sweep, df_rootkit, 
                      df_satan, df_smurf], ignore_index=True)

In [15]:
# verify append

final_df.tail(15)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
817536,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.241,0.095,0.001,0.095,0.0,0.0,0.0,0.0,0.0,smurf
817537,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.242,0.095,0.001,0.095,0.0,0.0,0.0,0.0,0.0,smurf
817538,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.243,0.095,0.001,0.095,0.0,0.0,0.0,0.0,0.0,smurf
817539,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.244,0.096,0.001,0.096,0.0,0.0,0.0,0.0,0.0,smurf
817540,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.245,0.096,0.001,0.096,0.0,0.0,0.0,0.0,0.0,smurf
817541,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.246,0.096,0.001,0.096,0.0,0.0,0.0,0.0,0.0,smurf
817542,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.247,0.097,0.001,0.097,0.0,0.0,0.0,0.0,0.0,smurf
817543,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.248,0.097,0.001,0.097,0.0,0.0,0.0,0.0,0.0,smurf
817544,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.249,0.098,0.001,0.098,0.0,0.0,0.0,0.0,0.0,smurf
817545,0.0,0.02,0.09,0.0,0.01032,0.0,0,0.0,0.0,0.0,...,0.250,0.098,0.001,0.098,0.0,0.0,0.0,0.0,0.0,smurf
